**Genes function prediction - RNN. Dataset - 70 000 sequences of DNA**

In [1]:
import re
import numpy as np
import torch
import string
from torch.nn.functional import cross_entropy
import matplotlib.pyplot as plt 
from sklearn.metrics import f1_score
from typing import Tuple, Optional, List
from torch.utils.data import Dataset, DataLoader

class ListDataset(Dataset):
    
    def __init__(self, new_data, classes): #przerobić new_data
        
        self.new_data = new_data
        self.classes = classes
        
    def __getitem__(self, ind):
        
        return self.new_data[ind], self.classes[ind]
    
    def __len__(self):
        return len(self.classes)

In [3]:
f = open('dna_70k_balanced.txt', "r")
data = f.read()
f.close()

elements = re.findall("([ATGC]+)\t+(\d+)", data)

sequences = [s for s, n in elements]
classes = [n for s, n in elements]
classes = [int(i) for i in classes]

In [4]:
def DNA_encoded(sequence):
    code = {"A": 0, "C": 1, "G": 2, "T": 3}
    fit = [code[x] for x in sequence]
    matrix = np.eye(4)[fit]
    return torch.FloatTensor(matrix)

In [5]:
new_data = [DNA_encoded(sequence) for sequence in sequences]

In [6]:
# split into train and test indices
test_frac = 0.1
n_test = int(test_frac * len(classes))
test_ind = np.random.choice(len(classes), size=n_test, replace=False)
train_ind = np.setdiff1d(np.arange(len(classes)), test_ind)

classes = torch.tensor(classes) #classes = classes.clone().detach()?
train_classes = classes[train_ind]

# calculate weights for BalancedSampler
uni, counts = np.unique(train_classes, return_counts=True)
weight_per_class = len(classes) / counts
weight = [weight_per_class[c] for c in train_classes]
# preapre the sampler
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights=weight, num_samples=len(weight)) 

train_dataset = ListDataset(new_data=[x for i, x in enumerate(new_data) if i in train_ind], classes=train_classes)
train_loader = DataLoader(train_dataset, shuffle=False, batch_size=1, sampler=sampler)

test_dataset = ListDataset(new_data=[x for i, x in enumerate(new_data) if i in test_ind], classes=classes[test_ind])
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=1)

In [9]:
class RNN(torch.nn.Module):
    
    def __init__(self, 
                 input_size: int,
                 hidden_size: int, 
                 output_size: int):
        """
        :param input_size: int
            Dimensionality of the input vector
        :param hidden_size: int
            Dimensionality of the hidden space
        :param output_size: int
            Desired dimensionality of the output vector
        """
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.input_to_hidden = torch.nn.Linear(input_size + hidden_size, hidden_size)

        self.hidden_to_output = torch.nn.Linear(hidden_size, output_size)
    
    # for the sake of simplicity a single forward will process only a single timestamp 
    def forward(self, 
                input: torch.tensor, 
                hidden: torch.tensor) -> Tuple[torch.tensor, torch.tensor]:
        """
        :param input: torch.tensor 
            Input tesnor for a single observation at timestep t
            shape [batch_size, input_size]
        :param hidden: torch.tensor
            Representation of the memory of the RNN from previous timestep
            shape [batch_size, hidden_size]
        """
        
        combined = torch.cat([input, hidden], dim=1) 
        hidden = torch.tanh(self.input_to_hidden(combined))
        output = self.hidden_to_output(hidden)
        
        return output, hidden
    
    def init_hidden(self, batch_size: int) -> torch.Tensor:
        """
        Returns initial value for the hidden state
        """
        return torch.zeros(batch_size, self.hidden_size, requires_grad=True)

In [ ]:
# initialize network and optimizer
rnn = RNN(4, 28, 7) #liczba liter, hidden, liczba klas
optimizer = torch.optim.SGD(rnn.parameters(), lr=0.01)   

epochs = 10

# main loop
for epoch in range(epochs):
    
    loss_buffer = []
    
    for i, (x, y) in enumerate(train_loader):  
        
        
        optimizer.zero_grad()
        # get initial hidden state
        hidden = rnn.init_hidden(x.shape[0])
        
        # get output for the sample, remember that we treat it as a sequence
        # so you need to iterate over the 2nd, time dimensiotn

        seq_len = x.shape[1]
        
        for t in range(seq_len): 
            x_t = x[:, t]
            output, hidden = rnn(input=x_t, hidden=hidden)
        
        loss = cross_entropy(output, y)
        loss.backward()
        optimizer.step()  
        
        loss_buffer.append(loss.item())
        
        if i % 1000 == 1:
            print(f"Epoch: {epoch} Progress: {100 * i/len(train_loader):2.0f}% Loss: {np.mean(loss_buffer):.3f}")
            loss_buffer = []


# evaluate on the test set
with torch.no_grad():
    ps = []
    ys = []
    correct = 0
    for i, (x, y) in enumerate(test_loader):
        ys.append(y.numpy())

        hidden = rnn.init_hidden(x.shape[0])
        seq_len = x.shape[1]
        
        for t in range(seq_len): 
            x_t = x[:, t]
            output, hidden = rnn(input=x_t, hidden=hidden)

        pred = output.argmax(dim=1)
        ps.append(pred.cpu().numpy())
    
    ps = np.concatenate(ps, axis=0)
    ys = np.concatenate(ys, axis=0)
    f1 = f1_score(ys, ps, average='weighted')
    
    print(f"Final F1 score: {f1:.2f}")
    assert f1 > 0.15, "You should get over 0.15 f1 score, try changing some hyperparams!"

Epoch: 0 Progress:  0% Loss: 1.950


In [7]:
gene_class = {'0': 'G protein coupled receptors', '1': 'Tyrosine Kinase', '2': 'Tyrosine Phosphatase', '3': 'Synthetase', '4': 'Synthase', '5':'Ion Chanel', '6': 'Transcription Factor'}

In [ ]:
def predict(seq: str, rnn: RNN):
    """Prints the name and model's top 3 predictions with scores"""
    hidden = rnn.init_hidden(1)

    for sequence in new_data:
      for tensor in sequence:
        output, hidden = rnn(tensor.unsqueeze(1).transpose(0, 1), hidden)
    
    topk = output.topk(3)
    
    first = gene_class[int(topk.indices[0][0])]
    second = gene_class[int(topk.indices[0][1])]
    third = gene_class[int(topk.indices[0][2])]
    
    print(f"    {first}: {topk.values[0][0]}")
    print(f"    {second}: {topk.values[0][1]}")
    print(f"    {third}: {topk.values[0][2]}")